In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import ChebConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)

In [3]:
fa_feature_path = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

fa_feature_path = "/home/snu/Downloads/Histogram_AD_FA_20bin_updated.npy"
Histogram_feature_AD_FA_array = np.load(fa_feature_path, allow_pickle=True)

X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_AD_FA_array.shape[0], dtype=np.int64)
])

num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (223, 180), Labels: (223,)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv

class Cheb_Supervised(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, K=3, activ="SELU"):
        super(Cheb_Supervised, self).__init__()
        self.conv1 = ChebConv(input_dim, hidden_dim, K=K)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)
        activations = {
            "SELU": F.selu,
            "SiLU": F.silu,
            "GELU": F.gelu,
            "RELU": F.relu,
            "ELU": F.elu
        }
        self.act = activations.get(activ, F.elu)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.act(x)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits


In [5]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [6]:
num_nodes, num_feats = X.shape
print(f"Number of features: {num_feats}")

Number of features: 180


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.8
feats_dim = num_feats
hidden_dim = 512
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 100

In [8]:
W = create_adj(X, alpha)
data = load_data(W, X).to(device)
A_tensor = torch.from_numpy(W).float().to(device)
print(data)

Data(x=[223, 180], edge_index=[2, 41689])


In [9]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.3, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

for fold, (train_idx, test_idx) in enumerate(sss.split(X, y), start=1):
    print(f"\n=== Fold {fold} ===")

    train_idx_t = torch.from_numpy(train_idx).long().to(device)
    y_train_tensor = torch.from_numpy(y[train_idx]).long().to(device)

    model = Cheb_Supervised(feats_dim, hidden_dim, num_classes, K=3, activ="SELU").to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()


    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss = ce_loss(logits[train_idx_t], y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: "
                  f"Loss={loss.item():.6f} | TrainAcc={acc_train:.4f}")


    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()  # Probability for class 1

    y_test = y[test_idx]
    y_pred_test = preds[test_idx]
    y_prob_test = probs[test_idx]


    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")


print("\n=== Average Results Across 20 Folds ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")


=== Fold 1 ===
Fold 1 Epoch 1: Loss=1.012253 | TrainAcc=0.3462
Fold 1 Epoch 100: Loss=0.198621 | TrainAcc=0.9423
Fold 1 Epoch 200: Loss=0.081927 | TrainAcc=0.9936
Fold 1 Epoch 300: Loss=0.033868 | TrainAcc=1.0000
Fold 1 Epoch 400: Loss=0.016530 | TrainAcc=1.0000
Fold 1 Epoch 500: Loss=0.009565 | TrainAcc=1.0000
Fold 1 Epoch 600: Loss=0.006025 | TrainAcc=1.0000
Fold 1 Epoch 700: Loss=0.004467 | TrainAcc=1.0000
Fold 1 Epoch 800: Loss=0.003224 | TrainAcc=1.0000
Fold 1 Epoch 900: Loss=0.002615 | TrainAcc=1.0000
Fold 1 Epoch 1000: Loss=0.001874 | TrainAcc=1.0000
Fold 1 Epoch 1100: Loss=0.001887 | TrainAcc=1.0000
Fold 1 Epoch 1200: Loss=0.001574 | TrainAcc=1.0000
Fold 1 Epoch 1300: Loss=0.001416 | TrainAcc=1.0000
Fold 1 Epoch 1400: Loss=0.001058 | TrainAcc=1.0000
Fold 1 Epoch 1500: Loss=0.000949 | TrainAcc=1.0000
Fold 1 Epoch 1600: Loss=0.000888 | TrainAcc=1.0000
Fold 1 Epoch 1700: Loss=0.000676 | TrainAcc=1.0000
Fold 1 Epoch 1800: Loss=0.000650 | TrainAcc=1.0000
Fold 1 Epoch 1900: Loss=0.0